### Pri tej nalogi je bolj zaplet

Bom probal čim bolje obrazložit kaj sem čaral in kako deluje.



#### Problemi, ki jih je treba rešit:

- Pregledati *vse* možne kombinacije barv, katera najhitreje pobarva matriko, na žalost, vzame preveč časa (10x10 matriko mi nikoli niti ni zaključilo, 6x6 pa lahko išče tudi dalj kot pol ure).
- Treba je zmanjšati število možnih potez v vsakem koraku, ampak paziti, da še vedno najde najboljšo rešitev.
- Treba je ugotoviti, kako sploh določiti *smiselne* poteze, da lahko ignoriramo tiste, ki se ne splačajo.
- Treba je Benotu sporočiti še kakšen način, da pohitri ta program, da mu program ne melje 15x15 matrike v nedogled.


Kako sem se jaz tega lotil bom opisal, ob kodi, kjer je rešitev. Kodo sem pisal v Angleščini ampak upam, da bo jasno.

#### Node

Imam razred **Node**, ki drži atribute:
 - **value**, ki predstavlja barvo elementa
 - **x** in **y**, ki sta pa pozicija elementa v matriki

In [27]:
class Node:
    def __init__(self, value, x, y):
        self.value = value
        self.x = x
        self.y = y

Hočem class, namesto integerja, zato da bom kasneje lažje preverjal med elementi matrike
in tukaj mi dosti olajša da lahko primerjam reference namesto int vrednost:

In [28]:
# Primer z Node razredom
node_1 = Node(value=3, x=1, y=1)
node_2 = Node(value=3, x=1, y=1)
print(f"{node_1 == node_2 = }")

# Primer z integerji
stev_1 = 3
stev_2 = 3

print(f"{stev_1 == stev_2 = }")

node_1 == node_2 = False
stev_1 == stev_2 = True


#### Path

In [29]:
class Path:
    def __init__(self, edge, options, colored, values):
        self.edge = edge
        self.options = options
        self.colored = colored
        self.len = len(self.colored)
        self.values = values

Ježeš kok argumentov in atributov v temle class-u!

Sicer bi jih lahko poimenoval malo bolje, ampak načeloma pomenijo:

- **edge**: to so vsa polja v matriki, ki se držijo trenutno obarvanih polj (tista, ki nismo mogli obarvati, ker so druge barve). Imenovan je "edge" zato ker je "rob" obarvanega dela.

- **options**: tukaj so shranjene vse barve (integerji), ki so v robu in so nam na voljo za naslednje barvanje (če imamo v "robu" samo barve 2, 3 in 5 bodo to tudi edine opcije, ki jh bomo imeli)

- **colored**: tukaj so vsi, ki sedijo zadaj na avtobusu - ne čak, tukaj so vsa polja, ki smo jih že obarvali in njih ne gledamo več. Ko so vsa polja v tem seznamu, vemo da smo obarvali vsa.

- **len**: je število že obarvanih polj, tu ga posebej shranim da lahko lažje poti sortiram glede na to koliko polj so obarvala


- **values**:tukaj je dictionary z key-value pari s katerim beležim koliko je še različnih barv v nepobarvanem delu matrike.


Dictionary je oblike {"barva": "število nepobarvanih polj te barve"} in če ima kakšna "barva" tukaj število polj 0 potem vem, da te barve ne bom več potreboval.


To je (trenutno) končna optimizacija v programu, in deluje tako, da v trenutni poti ne išče več rešitve če je:
**trenutni korak** + **število različnih barv še v matriki** >= **trenutno najboljši rešitvi**.


Ta del najbolj pospeši izvajanje, in ker sproti med barvanjem posodabljam vrednosti v dictionary, mi ni treba vsakič preiskati celotne matrike, katere barve so mi še ostale.

Ta del je zmanjšal 6x6 matrike iz dobre pol ure na manj kot desetinko sekunde, 10x10 matrike pa iz neskončno časa na par minut (sicer kakšne še zmeraj potrebujejo dalj kot pol ure).

# Glavni Class

The King Enchilada, kokr pravjo, kle nas bo mal infarkt, ampak bom probu pokomentirat kaj kje dela in pol spodi dodam mal bol podrobno :)

In [30]:
import math  # To rabim za neskončnost


class Solver:
    def __init__(self, matrika, values):  # Podam mu matriko in values, ki so isti kot v Path classu
        self.max_steps = math.inf  # trenutno najboljši rezultat shranjujem tukaj, začnem z neskončnostjo
        self.matrika = matrika
        self.size = len(matrika) * len(matrika[0])  # število elementov v matriki, da vem koliko jih rabim do konca
        self.beep = 0  # Število vseh korakov od vseh poskusov, s tem sem testiral optimizacije
        self.values = values  # values so pa isti valuesi kot v path classu

    def solve(self, edge, colored, options, step, values):
        """
        Rekurzivna funkcija, sprejme ista polja kot so v Path classu s to razliko da ima tudi trenutni korak.
        """
        step += 1  # trenutni korak povečam
        self.beep += 1  # beep prav tako haha

        """
        TALE if spodaj, pospeši izvedbo programa iz astronomskega časa nazaj na gastronomski čas!!!
        Če bi prej potrebovali do konca vesolja, zdej rabimo do konca ene pečene pice v povprečju!
        """
        if step + self.count_values(values=values) >= self.max_steps:
            # Potek sem opisal v Pathu ampak ja, če je korak + število različnih barv v nepobarvanem delu matrike večji ali enak najboljši rešitvi, prenehaj!
            return

        """
        Tale zakomentirani if je bil pa prvi poskus optimizacije, namesto vseh različnih barv v matriki gleda samo vse različne barve v robu.
        Čas izvajanja je bil okoli 100x daljši pri 6x6 matriki, pri 10x10 se mi pa nikoli ni dalo čakati na konec... Sprobajte oba :)
        """
        # if step + len(options) >= self.max_steps:
        #     return

        # Spodnji if konča izvajanje, če smo pobarvali celotno matriko, in če smo porabili manj korakov kot prej, jih določi za najboljši rezultat
        if len(colored) == self.size:
            if self.max_steps > step:
                self.max_steps = step
                # print(f"\n Current max step = {self.max_steps}")
            return

        # Če nas zgornji if-i niso vrgli ven iz te funkcije pa iščemo naprej:
        future_paths = []
        for option in options:  # za vsako opcijo preiščemo koliko polj obarva in shranimo v class Path
            new_edge, new_options, new_colored, new_values = self.dijsktra(edge=edge,
                                                                           # Funkcija za barvanje polj se imenuje dijsktra
                                                                           color=option,  # Ampak ni glih...
                                                                           colored=colored,
                                                                           # Jaz imam samo navado vse "breadth-first-search-e"
                                                                           values=values)  # imenovati dijsktra- v glavnem...

            path = Path(edge=new_edge, options=new_options, colored=new_colored,
                        values=new_values)  # rezultate od dijsktre shranim v Path
            future_paths.append(path)

        future_paths.sort(key=lambda x: x.len,
                          reverse=True)  # Sortiram po vrsti, da so tisti ki največ obarvajo na vrhu

        # In loopyloop, ki rekurzivno kliče funkcijo v kateri se trenutno nahajamo.
        for path in future_paths:
            self.solve(edge=path.edge, options=path.options, colored=path.colored, step=step, values=path.values)

        # To je ubistvu to, tale je bil glavni del rešitve!

    def dijsktra(self, edge, color, colored, values):
        """
        Dijsktra deluje tako da skopira pobarvana polja (colored), rob (edge) in values (values).
        Nato se sprehodi čez vsa polja v robu in pregleda njegove sosede.
        Polja doda v:
        1. "new_edge" če so različne barve kot izbrana barva (argument: color)
        2. "colored" če so iste barve kot izbrana barva, da se lahko sprehodi tudi čez njih

        Sosede od polj po katerih se sprehaja pa doda v:
        1. "new_edge" če so različne barve od izbrane
        2. "open_list" če so iste, da se lahko čez njih tudi sprehodi in pogleda še njegove sosede

        .
        .
        .
        huh, sej mogoče pa je čist dijsktra :P
        """
        colored = colored.copy()
        open_list = edge.copy()
        values = values.copy()
        colour_options = []
        new_edge = []
        while len(open_list) > 0:
            current_node = open_list.pop(0)
            if current_node.value != color:
                new_edge.append(current_node)
                if current_node.value not in colour_options:
                    colour_options.append(current_node.value)
                continue

            colored.append(current_node)
            values[color] -= 1
            cn_neighbours = self.get_neighbours(current_node, colored) # Funckija za prevert sosede
            for neighbour in cn_neighbours:
                if neighbour in open_list or neighbour in new_edge:
                    continue
                if neighbour.value == color:
                    open_list.append(neighbour)
                else:
                    new_edge.append(neighbour)
                    if neighbour.value not in colour_options:
                        colour_options.append(neighbour.value)

        return new_edge, colour_options, colored, values # vrne vse podatke, ki jih class Path in metoda solve() potrebujeta

    def get_neighbours(self, current_node: Node, colored):
        """
        V primerjavi z ostalimi, zelo enostavnejša metoda,
        vzame trenutno polje in pobarvana polja (celotna matrika pa je itak shranjena v samem Solve classu)

        Preveri gor, preveri dol, preveri levo in preveri desno in če polje obstaja in ni v pobarvanih ga dodaj na seznam veljavnih polj,
        to pa na koncu vrni.
        """
        valid_neighbours = []
        if current_node.x > 0:
            neighbour = self.matrika[current_node.y][current_node.x - 1]
            if neighbour not in colored:
                valid_neighbours.append(neighbour)
        if current_node.x < len(self.matrika[0]) - 1:
            neighbour = self.matrika[current_node.y][current_node.x + 1]
            if neighbour not in colored:
                valid_neighbours.append(neighbour)
        if current_node.y > 0:
            neighbour = self.matrika[current_node.y - 1][current_node.x]
            if neighbour not in colored:
                valid_neighbours.append(neighbour)
        if current_node.y < len(self.matrika) - 1:
            neighbour = self.matrika[current_node.y + 1][current_node.x]
            if neighbour not in colored:
                valid_neighbours.append(neighbour)
        return valid_neighbours

    def count_values(self, values):
        """
        Kle samo preštejem število barv z neničelnim številom polj, to je uno za obtimizacijo.
        """
        total = 0
        for key in values:
            if values[key] > 0:
                total += 1
        return total

#### Preživel smo :D
V spodnji funkciji preberem datoteko z matriko in shranim število polj na barvo v "values"

In [31]:
def read_input(filename):
    matrika = []
    values = {}
    with open(filename) as f:
        for y, line in enumerate(f.readlines()):
            row = []
            for x, value in enumerate(line.strip().split()):
                row.append(Node(value, x, y))
                if value not in values:
                    values[value] = 1
                else:
                    values[value] += 1
            matrika.append(row)
    return matrika, values


#### Probimo če dela

Najprej s 6x6 matrikami, te so shranjenje v mapi "light", z imeni 0.txt do 10.txt

In [32]:
import time # Dejmo še čas mert

In [33]:
for i in range(11):
    start = time.time()
    # Naložimo podatke iz datoteke
    matrika_light, values_light = read_input(f"./light/{i}.txt")

    # Ustvarimo instanco Solver classa
    solver = Solver(matrika=matrika_light, values=values_light)

    # Pokličemo Solver.solve() metodo in gasa
    solver.solve(edge=[solver.matrika[0][0]],
                 colored=[],
                 options=[solver.matrika[0][0].value],
                 step=-2,
                 values=values_light)
    end = time.time()
    print(f"Matrika Light: {i}, Rešitev: {solver.max_steps}; Čas: {end - start}")


Matrika Light: 0, Rešitev: 3; Čas: 0.00018739700317382812
Matrika Light: 1, Rešitev: 13; Čas: 0.002764463424682617
Matrika Light: 2, Rešitev: 11; Čas: 0.0007746219635009766
Matrika Light: 3, Rešitev: 12; Čas: 0.002373933792114258
Matrika Light: 4, Rešitev: 11; Čas: 0.0011196136474609375
Matrika Light: 5, Rešitev: 12; Čas: 0.0016138553619384766
Matrika Light: 6, Rešitev: 11; Čas: 0.001100778579711914
Matrika Light: 7, Rešitev: 15; Čas: 0.01175379753112793
Matrika Light: 8, Rešitev: 12; Čas: 0.001895904541015625
Matrika Light: 9, Rešitev: 13; Čas: 0.003589153289794922
Matrika Light: 10, Rešitev: 14; Čas: 0.0065271854400634766


#### 10x10 Matrike

Bodte pozorni na razlike v času izvajanja, pa 10x10 matrika se ne zdi tok grozn večja od 6x6 :D

In [34]:
for i in range(8):
    start = time.time()
    matrika_heavy, values_heavy = read_input(f"./heavy/{i}.txt")

    solver = Solver(matrika=matrika_heavy, values=values_heavy)
    solver.solve(edge=[solver.matrika[0][0]],
                 colored=[],
                 options=[solver.matrika[0][0].value],
                 step=-2,
                 values=values_heavy)
    end = time.time()
    print(f"Matrika Heavy: {i}, Rešitev: {solver.max_steps}; Čas: {end - start}")

Matrika Heavy: 0, Rešitev: 19; Čas: 82.18573689460754
Matrika Heavy: 1, Rešitev: 20; Čas: 59.49056577682495
Matrika Heavy: 2, Rešitev: 20; Čas: 48.59849143028259
Matrika Heavy: 3, Rešitev: 20; Čas: 114.55723571777344
Matrika Heavy: 4, Rešitev: 20; Čas: 133.46556448936462
Matrika Heavy: 5, Rešitev: 19; Čas: 10.675184488296509
Matrika Heavy: 6, Rešitev: 18; Čas: 38.48418045043945
Matrika Heavy: 7, Rešitev: 19; Čas: 43.895134687423706


Smešni časi, dejansko lahko rešiš 10 ali 20 tisoč 6x6 matrik prej kot eno 10x10 matriko :)

#### Če najdete kakšne izboljšave mi pa prosim javite! Hvala!